In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 1: Dataset Preprocessing (1 Done)

# Import necessary libraries
import os
import cv2
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
import random

In [ ]:
# Set paths
DATASET_CSV = '/content/HACER_dataset.csv'  # Path to your metadata CSV
VIDEO_DIR =  '/content/drive/MyDrive/HACER/'  # Path to your video files directory
PROCESSED_FRAMES_DIR = 'processed_frames'  # Directory to save extracted frames

In [ ]:
# Create directory for processed frames if it doesn't exist
os.makedirs(PROCESSED_FRAMES_DIR, exist_ok=True)

In [ ]:
# Load dataset CSV
data = pd.read_csv(DATASET_CSV)

In [ ]:
# Check dataset structure
print("Dataset loaded:")
print(data.head())

Dataset loaded:
          file_name  activity  emotion  video_index person   type
0  drink_angry_10_h         0        0           10      h   test
1   drink_angry_1_a         0        0            1      a  train
2   drink_angry_2_w         0        0            2      w  train
3   drink_angry_3_j         0        0            3      j   test
4   drink_angry_4_a         0        0            4      a  train


In [ ]:
# Function to extract frames from video
def extract_frames(video_path, output_dir, frame_count=10):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Processing video: {video_path}, Total frames: {total_frames}")
    frame_interval = max(1, total_frames // frame_count)
    frame_idx = 0

    while frame_idx < frame_count:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx * frame_interval)
        ret, frame = cap.read()
        if not ret:
            print(f"Failed to read frame at index {frame_idx}")
            break

        # Resize and save frame
        frame_resized = cv2.resize(frame, (224, 224))
        output_path = os.path.join(output_dir, f"frame_{frame_idx}.jpg")
        cv2.imwrite(output_path, frame_resized)
        print(f"Saved frame: {output_path}")
        frame_idx += 1

    cap.release()

In [ ]:
!ls /content/drive/MyDrive/HACER/


drink_angry_10_h.mp4	       putonjacket_neutral_9_k.mp4  standup_neutral_18_t.mp4
drink_angry_1_a.mp4	       putonjacket_sad_1_a.mp4	    standup_neutral_19_y.mp4
drink_angry_2_w.mp4	       putonjacket_sad_2_m.mp4	    standup_neutral_1_a.mp4
drink_angry_3_j.mp4	       putonjacket_sad_3_j.mp4	    standup_neutral_20_t.mp4
drink_angry_4_a.mp4	       putonjacket_sad_4_j.mp4	    standup_neutral_2_w.mp4
drink_angry_5_j.mp4	       putonjacket_sad_5_k.mp4	    standup_neutral_3_j.mp4
drink_angry_6_h.mp4	       putonjacket_sad_6_m.mp4	    standup_neutral_4_j.mp4
drink_angry_7_w.mp4	       read_angry_1_w.mp4	    standup_neutral_5_s.mp4
drink_angry_8_h.mp4	       read_angry_2_j.mp4	    standup_neutral_6_s.mp4
drink_angry_9_k.mp4	       read_angry_3_l.mp4	    standup_neutral_7_h.mp4
drink_disgust_10_m.mp4	       read_angry_4_w.mp4	    standup_neutral_8_a.mp4
drink_disgust_11_w.mp4	       read_angry_5_w.mp4	    standup_neutral_9_h.mp4
drink_disgust_12_h.mp4	       read_angry_6_l.mp4	    standup_sad_1

In [ ]:
data['video_filename'] = data['file_name'] + '.mp4'  # Add the correct extension


In [ ]:
missing_files = [row['video_filename'] for idx, row in data.iterrows() if not os.path.exists(os.path.join(VIDEO_DIR, row['video_filename']))]
print("Missing files:", missing_files)


Missing files: ['sitdown_happy_17_y.mp4', 'sitdown_happy_19_y.mp4']


In [ ]:
data = data[data['video_filename'].apply(lambda x: os.path.exists(os.path.join(VIDEO_DIR, x)))]
data.reset_index(drop=True, inplace=True)
print(f"Updated dataset size: {len(data)}")


Updated dataset size: 477


In [ ]:
for idx, row in data.iterrows():
    video_path = os.path.join(VIDEO_DIR, row['video_filename'])
    print(video_path, "exists" if os.path.exists(video_path) else "does not exist")


/content/drive/MyDrive/HACER/drink_angry_10_h.mp4 exists
/content/drive/MyDrive/HACER/drink_angry_1_a.mp4 exists
/content/drive/MyDrive/HACER/drink_angry_2_w.mp4 exists
/content/drive/MyDrive/HACER/drink_angry_3_j.mp4 exists
/content/drive/MyDrive/HACER/drink_angry_4_a.mp4 exists
/content/drive/MyDrive/HACER/drink_angry_5_j.mp4 exists
/content/drive/MyDrive/HACER/drink_angry_6_h.mp4 exists
/content/drive/MyDrive/HACER/drink_angry_7_w.mp4 exists
/content/drive/MyDrive/HACER/drink_angry_8_h.mp4 exists
/content/drive/MyDrive/HACER/drink_angry_9_k.mp4 exists
/content/drive/MyDrive/HACER/drink_disgust_10_m.mp4 exists
/content/drive/MyDrive/HACER/drink_disgust_11_w.mp4 exists
/content/drive/MyDrive/HACER/drink_disgust_12_h.mp4 exists
/content/drive/MyDrive/HACER/drink_disgust_13_k.mp4 exists
/content/drive/MyDrive/HACER/drink_disgust_14_t.mp4 exists
/content/drive/MyDrive/HACER/drink_disgust_15_s.mp4 exists
/content/drive/MyDrive/HACER/drink_disgust_16_t.mp4 exists
/content/drive/MyDrive/HAC

In [ ]:
# Extract frames for each video in the dataset
for idx, row in tqdm(data.iterrows(), total=len(data)):
    video_path = os.path.join(VIDEO_DIR, row['video_filename'])  # Replace 'video_filename' with correct column name
    output_dir = os.path.join(PROCESSED_FRAMES_DIR, f"video_{idx}")
    os.makedirs(output_dir, exist_ok=True)
    extract_frames(video_path, output_dir)

print("Frame extraction complete.")

  0%|          | 0/477 [00:00<?, ?it/s]

Processing video: /content/drive/MyDrive/HACER/drink_angry_10_h.mp4, Total frames: 141
Saved frame: processed_frames/video_0/frame_0.jpg
Saved frame: processed_frames/video_0/frame_1.jpg
Saved frame: processed_frames/video_0/frame_2.jpg
Saved frame: processed_frames/video_0/frame_3.jpg
Saved frame: processed_frames/video_0/frame_4.jpg
Saved frame: processed_frames/video_0/frame_5.jpg
Saved frame: processed_frames/video_0/frame_6.jpg
Saved frame: processed_frames/video_0/frame_7.jpg
Saved frame: processed_frames/video_0/frame_8.jpg


  0%|          | 1/477 [00:05<42:26,  5.35s/it]

Saved frame: processed_frames/video_0/frame_9.jpg
Processing video: /content/drive/MyDrive/HACER/drink_angry_1_a.mp4, Total frames: 320
Saved frame: processed_frames/video_1/frame_0.jpg
Saved frame: processed_frames/video_1/frame_1.jpg
Saved frame: processed_frames/video_1/frame_2.jpg
Saved frame: processed_frames/video_1/frame_3.jpg
Saved frame: processed_frames/video_1/frame_4.jpg
Saved frame: processed_frames/video_1/frame_5.jpg
Saved frame: processed_frames/video_1/frame_6.jpg
Saved frame: processed_frames/video_1/frame_7.jpg
Saved frame: processed_frames/video_1/frame_8.jpg


  0%|          | 2/477 [00:12<52:16,  6.60s/it]

...

Saved frame: processed_frames/video_474/frame_9.jpg
Processing video: /content/drive/MyDrive/HACER/write_neutral_8_a.mp4, Total frames: 269
Saved frame: processed_frames/video_475/frame_0.jpg
Saved frame: processed_frames/video_475/frame_1.jpg
Saved frame: processed_frames/video_475/frame_2.jpg
Saved frame: processed_frames/video_475/frame_3.jpg
Saved frame: processed_frames/video_475/frame_4.jpg
Saved frame: processed_frames/video_475/frame_5.jpg
Saved frame: processed_frames/video_475/frame_6.jpg
Saved frame: processed_frames/video_475/frame_7.jpg
Saved frame: processed_frames/video_475/frame_8.jpg


100%|█████████▉| 476/477 [30:45<00:04,  4.35s/it]

Saved frame: processed_frames/video_475/frame_9.jpg
Processing video: /content/drive/MyDrive/HACER/write_neutral_9_t.mp4, Total frames: 116
Saved frame: processed_frames/video_476/frame_0.jpg
Saved frame: processed_frames/video_476/frame_1.jpg
Saved frame: processed_frames/video_476/frame_2.jpg
Saved frame: processed_frames/video_476/frame_3.jpg
Saved frame: processed_frames/video_476/frame_4.jpg
Saved frame: processed_frames/video_476/frame_5.jpg
Saved frame: processed_frames/video_476/frame_6.jpg
Saved frame: processed_frames/video_476/frame_7.jpg
Saved frame: processed_frames/video_476/frame_8.jpg


100%|██████████| 477/477 [30:48<00:00,  3.88s/it]

Saved frame: processed_frames/video_476/frame_9.jpg
Frame extraction complete.


In [ ]:
# Split dataset into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Save train/test splits
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

print("Train/test split complete. Train and test CSVs saved.")

Train/test split complete. Train and test CSVs saved.


In [ ]:
# Step 2: Data Augmentation (2 Done)
from google.colab import files
uploaded = files.upload()

# After uploading, access the file
TRAIN_CSV = '/content/train_videos.csv'
train_data = pd.read_csv(TRAIN_CSV)


Saving train_videos.csv to train_videos.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test_videos.csv to test_videos.csv


In [ ]:
# Paths to the data splits
TRAIN_CSV = '/content/train_videos.csv'
AUGMENTED_FRAME_DIR = '/content/augmented_frames/'  # Directory for augmented data
os.makedirs(AUGMENTED_FRAME_DIR, exist_ok=True)

In [ ]:
print("Training data path:", TRAIN_CSV)


Training data path: /content/train_videos.csv


In [ ]:
# Load train data
train_data = pd.read_csv(TRAIN_CSV)
print("Train data loaded:")
print(train_data.head())

Train data loaded:
                                  video_path  label
0  /content/drive/MyDrive/HACER_frames/117_3      3
1  /content/drive/MyDrive/HACER_frames/137_0      0
2  /content/drive/MyDrive/HACER_frames/209_0      0
3  /content/drive/MyDrive/HACER_frames/409_0      0
4  /content/drive/MyDrive/HACER_frames/148_2      2


In [ ]:
# Initialize ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)


In [ ]:
# Function to apply augmentation to frames
def augment_frames(video_dir, save_dir, augment_count=5):
    frame_files = [os.path.join(video_dir, f) for f in os.listdir(video_dir) if f.endswith('.jpg')]
    print(f"Augmenting frames in: {video_dir}")
    for frame_file in frame_files:
        frame = cv2.imread(frame_file)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB (for TensorFlow)
        frame = cv2.resize(frame, (224, 224))
        frame = frame.reshape((1,) + frame.shape)  # Add batch dimension

        # Generate augmentations
        i = 0
        for batch in datagen.flow(frame, batch_size=1):
            save_path = os.path.join(save_dir, f"{os.path.basename(frame_file).split('.')[0]}_aug_{i}.jpg")
            augmented_frame = batch[0].astype('uint8')
            cv2.imwrite(save_path, cv2.cvtColor(augmented_frame, cv2.COLOR_RGB2BGR))  # Convert RGB back to BGR
            print(f"Saved augmented frame: {save_path}")
            i += 1
            if i >= augment_count:  # Stop after generating augment_count frames
                break

In [ ]:
# Load the CSV file
train_data = pd.read_csv(TRAIN_CSV)

# Print the column names to verify
print(train_data.columns)


Index(['video_path', 'label'], dtype='object')


In [ ]:
# Augment train frames
for idx, row in train_data.iterrows():
    video_frame_dir = row['video_path']
    label = row['label']
    augmented_video_dir = os.path.join(AUGMENTED_FRAME_DIR, f"{idx}_{label}")
    os.makedirs(augmented_video_dir, exist_ok=True)
    augment_frames(video_frame_dir, augmented_video_dir)

print("Data augmentation complete. Augmented frames saved.")

Augmenting frames in: /content/drive/MyDrive/HACER_frames/117_3
Augmenting frames in: /content/drive/MyDrive/HACER_frames/137_0
Augmenting frames in: /content/drive/MyDrive/HACER_frames/209_0
Augmenting frames in: /content/drive/MyDrive/HACER_frames/409_0
Augmenting frames in: /content/drive/MyDrive/HACER_frames/148_2
Augmenting frames in: /content/drive/MyDrive/HACER_frames/312_3
Augmenting frames in: /content/drive/MyDrive/HACER_frames/22_1
Augmenting frames in: /content/drive/MyDrive/HACER_frames/314_3
Augmenting frames in: /content/drive/MyDrive/HACER_frames/46_2
Augmenting frames in: /content/drive/MyDrive/HACER_frames/398_4
Augmenting frames in: /content/drive/MyDrive/HACER_frames/284_0
Augmenting frames in: /content/drive/MyDrive/HACER_frames/204_4
Augmenting frames in: /content/drive/MyDrive/HACER_frames/182_2
Augmenting frames in: /content/drive/MyDrive/HACER_frames/395_4
Augmenting frames in: /content/drive/MyDrive/HACER_frames/57_3
Augmenting frames in: /content/drive/MyDriv

In [ ]:
# Step 3: Feature Extraction using CNN (3 Done)
# Paths
AUGMENTED_FRAME_DIR = '/content/augmented_frames/'  # Directory with augmented frames
FEATURES_DIR = '/content/features/'  # Directory to save extracted features
os.makedirs(FEATURES_DIR, exist_ok=True)

In [ ]:
# Load the pre-trained CNN (ResNet50)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)
print("ResNet50 model loaded.")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
ResNet50 model loaded.


In [ ]:
# Function to extract features for a video
def extract_features_from_video(video_dir, save_path):
    frame_files = sorted([os.path.join(video_dir, f) for f in os.listdir(video_dir) if f.endswith('.jpg')])
    print(f"Extracting features from video: {video_dir}")
    features_list = []
    for frame_file in frame_files:
        frame = cv2.imread(frame_file)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (224, 224))
        frame = preprocess_input(np.expand_dims(frame, axis=0))  # Preprocess for ResNet50
        features = model.predict(frame)
        features_list.append(features.squeeze())
    # Save the features for the video
    np.save(save_path, np.array(features_list))
    print(f"Features saved at: {save_path}")

In [ ]:
# Extract features for all augmented videos
video_folders = [os.path.join(AUGMENTED_FRAME_DIR, f) for f in os.listdir(AUGMENTED_FRAME_DIR)]
for video_folder in video_folders:
    video_id = os.path.basename(video_folder)
    save_path = os.path.join(FEATURES_DIR, f"{video_id}_features.npy")
    extract_features_from_video(video_folder, save_path)

print("Feature extraction complete. Features saved in:", FEATURES_DIR)

Extracting features from video: /content/augmented_frames/330_3
Features saved at: /content/features/330_3_features.npy
Extracting features from video: /content/augmented_frames/33_0
Features saved at: /content/features/33_0_features.npy
Extracting features from video: /content/augmented_frames/294_4
Features saved at: /content/features/294_4_features.npy
Extracting features from video: /content/augmented_frames/200_3
Features saved at: /content/features/200_3_features.npy
Extracting features from video: /content/augmented_frames/245_2
Features saved at: /content/features/245_2_features.npy
Extracting features from video: /content/augmented_frames/309_2
Features saved at: /content/features/309_2_features.npy
Extracting features from video: /content/augmented_frames/300_2
Features saved at: /content/features/300_2_features.npy
Extracting features from video: /content/augmented_frames/77_2
Features saved at: /content/features/77_2_features.npy
Extracting features from video: /content/aug

In [ ]:
# Step 4: Temporal Analysis using RNN (4 Done)
# Paths
FEATURES_DIR = '/content/features/'  # Directory with saved features
TRAIN_CSV = '/content/train_videos.csv'
TEST_CSV = '/content/test_videos.csv'

In [ ]:
# Load train and test data
train_data = pd.read_csv(TRAIN_CSV)
test_data = pd.read_csv(TEST_CSV)
print("Train and test data loaded.")

Train and test data loaded.


In [ ]:
# Encode labels
encoder = LabelEncoder()
train_labels = encoder.fit_transform(train_data['label'])
test_labels = encoder.transform(test_data['label'])

In [ ]:
# Convert labels to categorical (for softmax output)
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
import os
print(os.listdir('/content/features/'))


['295_2_features.npy', '101_2_features.npy', '3_0_features.npy', '228_3_features.npy', '114_4_features.npy', '232_3_features.npy', '199_2_features.npy', '206_4_features.npy', '119_3_features.npy', '120_2_features.npy', '289_0_features.npy', '356_3_features.npy', '202_4_features.npy', '157_4_features.npy', '347_4_features.npy', '319_0_features.npy', '140_2_features.npy', '135_2_features.npy', '159_2_features.npy', '130_0_features.npy', '195_4_features.npy', '274_0_features.npy', '21_3_features.npy', '313_4_features.npy', '285_0_features.npy', '73_2_features.npy', '279_2_features.npy', '247_0_features.npy', '171_4_features.npy', '301_0_features.npy', '320_3_features.npy', '365_0_features.npy', '85_0_features.npy', '235_2_features.npy', '305_3_features.npy', '134_2_features.npy', '219_3_features.npy', '326_2_features.npy', '80_2_features.npy', '299_4_features.npy', '92_3_features.npy', '5_3_features.npy', '189_2_features.npy', '213_0_features.npy', '54_0_features.npy', '53_1_features.npy'

In [ ]:
feature_path = os.path.join(FEATURES_DIR, f"{os.path.basename(row['video_path']).replace('.mp4', '')}_features.npy")


In [ ]:
print(train_data['video_path'].head())


0    /content/drive/MyDrive/HACER_frames/117_3
1    /content/drive/MyDrive/HACER_frames/137_0
2    /content/drive/MyDrive/HACER_frames/209_0
3    /content/drive/MyDrive/HACER_frames/409_0
4    /content/drive/MyDrive/HACER_frames/148_2
Name: video_path, dtype: object


In [ ]:
print(f"Expected feature path: {feature_path}")


Expected feature path: /content/features/102_2_features.npy


In [ ]:
import os
import numpy as np

FEATURES_DIR = '/content/features/'  # Directory with saved features

# Load features
def load_features(data):
    features = []
    feature_shape = None  # To store the shape of the first valid feature file
    valid_indices = []  # Keep track of valid rows

    for idx, row in data.iterrows():
        # Use os.path.basename to get only the filename from video_path
        video_filename = os.path.basename(row['video_path'])
        # Remove the extension (.mp4) and construct the feature filename
        feature_filename = video_filename.replace(".mp4", "") + "_features.npy"
        feature_path = os.path.join(FEATURES_DIR, feature_filename)

        # Check if the feature file exists
        if os.path.exists(feature_path):
            video_features = np.load(feature_path)
            if feature_shape is None:
                feature_shape = video_features.shape  # Get the shape of the first valid feature
            features.append(video_features)
            valid_indices.append(idx)  # Keep track of valid rows
            print(f"Loaded features from: {feature_path}")
        else:
            print(f"Warning: Feature file not found for {video_filename}. Skipping.")

    # Create a new DataFrame with only valid rows
    valid_data = data.iloc[valid_indices]  # Select only rows with valid features
    return np.array(features), valid_data

# Load data (replace these with actual DataFrame loading if necessary)
x_train, valid_train_data = load_features(train_data)
x_test, valid_test_data = load_features(test_data)

print(f"x_train shape: {x_train.shape}, x_test shape: {x_test.shape}")


In [ ]:
import os
print(os.listdir('/content/features/'))  # List all files in the features directory


['295_2_features.npy', '101_2_features.npy', '3_0_features.npy', '228_3_features.npy', '114_4_features.npy', '232_3_features.npy', '199_2_features.npy', '206_4_features.npy', '119_3_features.npy', '120_2_features.npy', '289_0_features.npy', '356_3_features.npy', '202_4_features.npy', '157_4_features.npy', '347_4_features.npy', '319_0_features.npy', '140_2_features.npy', '135_2_features.npy', '159_2_features.npy', '130_0_features.npy', '195_4_features.npy', '274_0_features.npy', '21_3_features.npy', '313_4_features.npy', '285_0_features.npy', '73_2_features.npy', '279_2_features.npy', '247_0_features.npy', '171_4_features.npy', '301_0_features.npy', '320_3_features.npy', '365_0_features.npy', '85_0_features.npy', '235_2_features.npy', '305_3_features.npy', '134_2_features.npy', '219_3_features.npy', '326_2_features.npy', '80_2_features.npy', '299_4_features.npy', '92_3_features.npy', '5_3_features.npy', '189_2_features.npy', '213_0_features.npy', '54_0_features.npy', '53_1_features.npy'

In [ ]:
print(train_data.head())  # Check the first few rows of train_data
print(test_data.head())   # Check the first few rows of test_data


                                  video_path  label
0  /content/drive/MyDrive/HACER_frames/117_3      3
1  /content/drive/MyDrive/HACER_frames/137_0      0
2  /content/drive/MyDrive/HACER_frames/209_0      0
3  /content/drive/MyDrive/HACER_frames/409_0      0
4  /content/drive/MyDrive/HACER_frames/148_2      2
                                  video_path  label
0   /content/drive/MyDrive/HACER_frames/93_2      2
1  /content/drive/MyDrive/HACER_frames/320_3      3
2   /content/drive/MyDrive/HACER_frames/55_3      3
3   /content/drive/MyDrive/HACER_frames/30_2      2
4  /content/drive/MyDrive/HACER_frames/449_0      0


In [ ]:
# Define RNN Model
rnn_model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(None, x_train.shape[-1])),
    Dropout(0.3),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(train_labels.shape[1], activation='softmax')  # Number of emotion classes
])

rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("RNN model compiled.")

RNN model compiled.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Train the RNN
history = rnn_model.fit(
    x_train, train_labels,
    validation_split=0.2,
    epochs=25,
    batch_size=32,
    verbose=1
)

In [ ]:
# Evaluate on the test set
test_loss, test_accuracy = rnn_model.evaluate(x_test, test_labels, verbose=0)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
# Save the trained RNN model
MODEL_PATH = '/content/emotion_recognition_model.h5'
rnn_model.save(MODEL_PATH)
print(f"Model saved at {MODEL_PATH}")

In [ ]:
# Visualize Training History
import matplotlib.pyplot as plt

# Plot accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Test the Model with Predictions
def predict_video(video_path):
    features = np.load(video_path)
    predictions = rnn_model.predict(np.expand_dims(features, axis=0))
    predicted_class = np.argmax(predictions)
    return encoder.inverse_transform([predicted_class])[0]

In [ ]:
# Example prediction
example_video_path = os.path.join(FEATURES_DIR, os.listdir(FEATURES_DIR)[0])
print(f"Predicted emotion: {predict_video(example_video_path)}")
